## Julia / Yao implementation of a VQE for finding the ground state energy of H2.

For this implementation I use QuAlgorithmZoo.jl's VQE_openfermion found [here](https://github.com/QuantumBFS/QuAlgorithmZoo.jl/blob/master/examples/VQE_openfermion/main.jl). The original implementation uses a VQE to find the lowest energy state at specific bond lengths. 

The main change is the golden section search I use to find the ground state energy of the system. 

In [2]:
# Full code can be found at: https://github.com/QuantumBFS/QuAlgorithmZoo.jl/blob/master/examples/VQE_openfermion/main.jl 

using Yao
using Yao.EasyBuild

# Here we we invoke [OpenFermion](https://github.com/quantumlib/OpenFermion) in Python
# to get the hamiltonian under [Jordan-Weigner transformation](https://en.wikipedia.org/wiki/Jordan%E2%80%93Wigner_transformation)
# (with 2 electrons and 4 sto-3g basis set orbitagls).

using PyCall

py"""
import numpy as np
from openfermion.config import *
from openfermion import MolecularData
from openfermion.transforms import jordan_wigner
from openfermionpyscf import run_pyscf

def make_H2_mol(bond_len):
    ## Returns:
    ##     molecule(openfermion.hamiltonians.MolecularData):
    ## MolecularData for H2 with certain bond length
    
    ## Create molecule
    atom_1 = 'H'
    atom_2 = 'H'
    basis = 'sto-3g'
    multiplicity = 1
    charge = 0
    coordinate_1 = (0.0, 0.0, 0.0)
    coordinate_2 = (0.0, 0.0, bond_len)
    geometry = [(atom_1, coordinate_1), (atom_2, coordinate_2)]
    molecule = MolecularData(geometry, basis, multiplicity,
    charge, description=str(bond_len))
    molecule = run_pyscf(molecule,run_scf=1,run_ccsd=1,run_fci=1)
    return molecule

def get_hamiltonian(bond_len):
    ## get the coefficients and pauli terms of Hamiltonian of H_2 with given bond length
    ## Returns:
    ##     (PyArray,PyArray)
    molecule = make_H2_mol(bond_len)
    qubit_hamiltonian = jordan_wigner(molecule.get_molecular_hamiltonian())
    qubit_hamiltonian.compress()
    return list(qubit_hamiltonian.terms.keys()),list(qubit_hamiltonian.terms.values())

"""

# Define the function to transfer OpenFermion hamiltonian to Yao circuit

function get_fermion_hamiltonian(n::Int,terms::Array,coefs::Vector)
    gates=Dict("Z"=>Z,"X"=>X,"Y"=>Y)
    to_pauli(t::Tuple{Int,String})=put(n,t[1]+1=>get(gates,t[2],ErrorException("Invalid")))
    return sum(coefs[2:end].*map(x->reduce(*,map(to_pauli,x)),terms[2:end]))
end

# Use VQE code given in the previous Yao example

using Flux: Optimise

function train!(circ, hamiltonian; optimizer, niter::Int=2500,verbose::Bool=true)
     params = parameters(circ)
     dispatch!(circ, :random)
     for i=1:niter
         _, grad = expect'(hamiltonian, zero_state(nqubits(circ)) => circ)
         Optimise.update!(optimizer, params, grad)
         dispatch!(circ, params)
         if verbose
            println("Energy = 
            $(expect(hamiltonian, zero_state(nqubits(hamiltonian)) |> circ) |> real)")
         end
     end
     return expect(hamiltonian, zero_state(nqubits(hamiltonian)) |> circ)
end

# Train VQE to search ground energy on various bond lengths.

pes = Vector{Float64}()

using Optim
using LinearAlgebra

function vqe_calculation(bond_len)
    terms, coefs = py"get_hamiltonian"(bond_len)

    e = coefs[1]
    
    h = get_fermion_hamiltonian(4,terms,coefs)
    c = variational_circuit(4)
    
    emin_vqe = train!(c, h; optimizer=Optimise.ADAM(0.001, (0.9, 0.99)),verbose=false)
    # println("Estimated minimum energy at bond length $(bond_len): $(e+emin_vqe|>real)")
    push!(pes,e+emin_vqe|>real)
    
    emin = eigvals(Matrix(mat(h)))[1]
    @assert isapprox(emin, emin_vqe, atol=1e-3)

    return e+emin_vqe|>real
end 

# Using a search function GoldenSection() to find the lowest energy state of the system. 
# Optimize: https://julianlsolvers.github.io/Optim.jl/stable/user/minimization/ 

function vqe(iter)
    minimizer_result::Float64 = 0
    minimum_result::Float64 = 0

    for i in 1:iter
        print("Iteration: $(i) process result: ")
        @time results = optimize(vqe_calculation, 0.5, 1, GoldenSection();)

        minimizer_result += results.minimizer
        minimum_result += results.minimum
    end
    print("End results: \n")

    print("Minimum bond length: $(minimizer_result/iter)\n")
    print("Minimum energy: $(minimum_result/iter)\n")

end

#@time results = 
@time vqe(5)

#print(results)

# Plot the potential energy surface (PES)

# using Plots

# Plots.plot(bond_len_overtime,pes, markershape = :circle, linestyle = :dash, label = "VQE",ylabel = "Ground State Energy (Hartree)", xlabel = "Bond Length (Å) ")

Iteration: 1 process result:   5.155237 seconds (163.50 M allocations: 5.745 GiB, 4.49% gc time, 0.82% compilation time)
Iteration: 2 process result:   5.150254 seconds (163.42 M allocations: 5.740 GiB, 4.69% gc time)
Iteration: 3 process result:   5.544472 seconds (163.42 M allocations: 5.740 GiB, 4.83% gc time)
Iteration: 4 process result:   5.583342 seconds (163.42 M allocations: 5.740 GiB, 4.84% gc time)
Iteration: 5 process result:   5.316461 seconds (163.42 M allocations: 5.740 GiB, 4.96% gc time)
End results: 
Minimum bond length: 0.7348345554452321
Minimum energy: -1.1373060402199182
 26.750889 seconds (817.20 M allocations: 28.703 GiB, 4.76% gc time, 0.16% compilation time)
